# Face detection and recognition training pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and recogition on an image dataset using an Inception Resnet V1 pretrained on the VGGFace2 dataset.

The following Pytorch methods are included:
* Datasets
* Dataloaders
* GPU/CPU processing

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, prewhiten, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import multiprocessing as mp
import os

#### Define run parameters

In [7]:
data_dir = '../../../data/vggface2/train'
batch_size = 16
epochs = 15

#### Determine if an nvidia GPU is available

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

Default params shown for illustration, but not needed. Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [4]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtained cropped faces.

In [12]:
dataset = datasets.ImageFolder(data_dir)
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=lambda x: x[0], num_workers=mp.cpu_count(), shuffle=False)

for i, (x, y) in enumerate(loader):
    print(f'\rImages processed: {i + 1} of {len(loader)}', end='')
    save_dir = os.path.join(data_dir + '_cropped', dataset.idx_to_class[y])
    os.makedirs(save_dir, exist_ok=True)
    filename = f'{len(os.listdir(save_dir)):05n}.png'
    mtcnn(x, save_path=os.path.join(save_dir, filename))

Images processed: 6353 of 6353

#### Define Inception Resnet V1 module

Set classify=True for classifier.

See `help(InceptionResnetV1)` for more details.

In [13]:
resnet = InceptionResnetV1(
    pretrained='vggface2',
    classify=True,
    num_classes=len(dataset.class_to_idx)
).to(device)

#### Define optimizer, scheduler, dataset, and dataloader

In [14]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    prewhiten
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=mp.cpu_count(),
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=mp.cpu_count(),
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [15]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [ ]:
print(f'\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device
)

for epoch in range(epochs):
    print(f'\n\nEpoch {epoch + 1}/{epochs}')
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device
    )



Initial
----------
Eval  |    80/80   | loss:    2.9421 | fps:    7.6358 | acc:    0.0602   


Epoch 1/15
----------
Train |   317/317  | loss:    1.9690 | fps:    2.4324 | acc:    0.5260   
Eval  |    80/80   | loss:    1.4802 | fps:    8.2792 | acc:    0.5591   


Epoch 2/15
----------
Train |   317/317  | loss:    1.0367 | fps:    2.4487 | acc:    0.7467   
Eval  |    80/80   | loss:    0.8572 | fps:    8.0474 | acc:    0.7799   


Epoch 3/15
----------
Train |   124/317  | loss:    0.6837 | fps:    2.4360 | acc:    0.8362   